* 베이즈 정리는 머신러닝 분류(classification) 문제를 해결하는데 사용될 수 있다.

# 1. 과일 선택 문제
어떤 과일 가게에서 사과와 오렌지를 팔고 있다. 사과와 오렌지는 Red 농장과 Blue 농장에서 공급받는다.  
어느 날 Red 농장에서 붉은 상자에 사과 2개, 오렌지 6개를 담아서 보냈다.  
Blue 농장에서는 푸른 상자에 사과 3개, 오렌지 1개를 담아서 공급했다.  
손님이 와서 오렌지 하나를 선택했다.(X = 오렌지)  
이 오렌지는 Red 농장에서 온 것일까(Y = Red), 아니면 Blue 농장에서 온 것일까(Y = Blue)?  
만약 Red 농장과 Blue 농장 중 어느 농장을 선택하겠는가?

------------
답은 오렌지가 더 많은 상자를 공급한 Red 농장일 가능성이 높은 것이다.  
그 가능성 즉, 확률을 수치화보면

#### 용어 정리
* 손님이 선택한 과일이 사과인 사건 X = A
* 손님이 선택한 과일이 오렌지인 사건 X = O
* 손님이 선택한 과일이 Red 농장에서 왔을 사건 Y = R
* 손님이 선택한 과일이 Blue 농장에서 왔을 사건 Y = B

#### 우리가 원하는 확률
"손님이 오렌지를 선택한 경우에 원산지가 Red 농장일 확률"이므로 조건부 확률이다.
$$ P(Y = R|X = O) $$

#### 위 조건부 확률값을 구하기 위해 구할 확률
* 전체 샘플 12개 중 Red 농장에서 온 샘플이 8개 이므로
$$ P(Y = R) = 8/(8 + 4) = 8/12 $$
* Red 농장의 샘플 8개 중 오렌지가 6개 이므로
$$ P(X = O|Y = R) = 6/(2 + 6) = 6/8 $$

#### 조건부 확률 계산
$$ P(Y = R|X = O) = P(X = O|Y = R)P(Y = R)/P(X = O) $$
$$ = P(X = O|Y = R)P(Y = R)/\{P(X = O|Y = R)P(Y = R) + P(X = O|Y = B)P(Y = B)\} $$
$$ = (6/8 * 8/12)/(6/8 * 8/12 + 1/4*4/12) $$
$$ = (1/2)/(1/2 + 1/12) = 6/7 $$

#### Blue 농장인 확률
농장이 두 개 뿐이므로 원상지가 Blue 농장인 확률은 1 - P(Y = R|X = O)이다.
$$ P(Y = B|X = O) = 1 - P(Y = R|X = O) = 1/7 $$

#### 답 (최종 확률)
* Red 농장일 확률 : 6/7
* Blue 농장일 확률 : 1/7
* 예측 결과는 Red 농장이다.

------
이런 방식으로 분류 문제를 풀기 위해서는 각각의 출력 카테고리 Y에 대한 특징값 X의 분포, 즉 likehood를 알고 있어야 한다.  
#### 생성론적 방법(generative method)
이렇게 베이즈 정리와 likehood를 이용하여 각각의 Y값에 대한 확률 값을 모두 구한 다음, 가장 확률 값이 높은 Y값을 선택하여 분류 문제를 푸는 방법

#### pgmpy를 사용한 문제 풀이

In [1]:
from pgmpy.factors.discrete import TabularCPD

# 확률 변수 Y = 0인 경우 : Red 농장
# Y = 1인 경우 : Blue 농장
cpd_Y = TabularCPD('Y', 2, [[8/12, 4/12]])
print(cpd_Y)

╒═════╤══════════╕
│ Y_0 │ 0.666667 │
├─────┼──────────┤
│ Y_1 │ 0.333333 │
╘═════╧══════════╛


In [3]:
# 확률 변수 X = 0인 경우 : 오렌지
# X = 1인 경우 : 사과
import numpy as np

cpd_XY = TabularCPD('X', 2, np.array([[6/8, 1/4], [2/8, 3/4]]),
                   evidence=['Y'], evidence_card=[2])
print(cpd_XY)

╒═════╤══════╤══════╕
│ Y   │ Y_0  │ Y_1  │
├─────┼──────┼──────┤
│ X_0 │ 0.75 │ 0.25 │
├─────┼──────┼──────┤
│ X_1 │ 0.25 │ 0.75 │
╘═════╧══════╧══════╛


In [4]:
# 베이지안 네트워크 모형으로 사후확률을 계산하면 0.1429 = 1/7이 나온다.
from pgmpy.models import BayesianModel

model = BayesianModel([('Y', 'X')])
model.add_cpds(cpd_Y, cpd_XY)
model.check_model()

True

In [5]:
from pgmpy.inference import VariableElimination

infer = VariableElimination(model)
posterior = infer.query(['Y'], evidence={'X': 0})
print(posterior['Y'])

╒═════╤══════════╕
│ Y   │   phi(Y) │
╞═════╪══════════╡
│ Y_0 │   0.8571 │
├─────┼──────────┤
│ Y_1 │   0.1429 │
╘═════╧══════════╛
